In [ ]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender

from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython
from MatrixFactorization.PureSVD import PureSVDRecommender

from Base.NonPersonalizedRecommender import TopPop, Random

from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender

from KNN.ItemKNNScoresHybridRecommender import ItemKNNScoresHybridRecommender
from KNN.ItemKNNScoresHybridRecommender3 import ItemKNNScoresHybridRecommender3

from KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from KNN.ItemKNNSimilarityHybridRecommender3 import ItemKNNSimilarityHybridRecommender3
from KNN.ItemKNNSimilarityHybridRecommender4 import ItemKNNSimilarityHybridRecommender4

from KNN.UserKNNCBFRecommender import UserKNNCBFRecommender


#from GraphBased.RP3betaRecommender import RP3betaRecommender
from GraphBased.P3alphaRecommender import P3alphaRecommender

In [3]:
import traceback, os
import scipy.sparse
import pandas as pd
import numpy as np

URM_all = scipy.sparse.load_npz('./Matrix/URM_all_matrix.npz')
ICM_all = scipy.sparse.load_npz('./Matrix/ICM_all_matrix.npz')
URM_train = scipy.sparse.load_npz('./Matrix/URM_train_matrix.npz')
URM_test = scipy.sparse.load_npz('./Matrix/URM_test_matrix.npz')
#UCM_all = scipy.sparse.load_npz('./Matrix/UCM_all_matrix.npz')
#UCM_train = scipy.sparse.load_npz('UCM_train_matrix.npz')



#Load playlistsIDS
train_df= pd.read_csv('all/train.csv',low_memory = False)
all_userID=train_df.playlist_id.unique()

In [ ]:
CFrecommender = ItemKNNCFRecommender(URM_train)
CFrecommender.fit()

SLIMrecommender = SLIM_BPR_Cython(URM_train)
SLIMrecommender.fit()

CBFrecommender = ItemKNNCBFRecommender(ICM_all,URM_train)
CBFrecommender.fit()

SLIMErecommender = SLIMElasticNetRecommender(URM_train)
SLIMErecommender.fit()

UCFRecommender = UserKNNCFRecommender(URM_train)
UCFRecommender.fit()

In [4]:
SVDrecommender = PureSVDRecommender (URM_train)
SVDrecommender.fit(num_factors=1000,n_iter=40)
result = SVDrecommender.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))

PureSVDRecommender Computing SVD decomposition...
PureSVDRecommender Computing SVD decomposition... Done!
SequentialEvaluator: Processed 9676 ( 100.00% ) in 10.93 seconds. Users per second: 885
Recommender MAP is= 0.05963453726709093


In [8]:
UCFRecommender = UserKNNCFRecommender(URM_train)
UCFRecommender.fit()
result = UCFRecommender.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))

Similarity column 41798 ( 83 % ), 1378.20 column/sec, elapsed time 0.51 min
Similarity column 50446 ( 100 % ), 1391.59 column/sec, elapsed time 0.60 min
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.51 seconds. Users per second: 1755
Recommender MAP is= 0.08169599022240999


In [6]:
p3Arecommender=P3alphaRecommender(URM_train)
p3Arecommender.fit()
result = p3Arecommender.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))

SequentialEvaluator: Processed 9676 ( 100.00% ) in 7.00 seconds. Users per second: 1381
Recommender MAP is= 0.08152686246076078


In [ ]:
#a is= 0.8446051373150227 ,b is= 0.5195150893129805, c is= 0.1020614619309752
Hrecommender=ItemKNNScoresHybridRecommender3(URM_train,UCFRecommender, p3Arecommender,SVDrecommender)
#alphaList = [(0.5, 0.5, 0.2), (0.5, 0.4, 0.1), (0.5, 0.5, 0.3), (0.5, 0.3, 0.1), (0.4, 0.3, 0.2), (0.4, 0.2, 0.2), (0.6, 0.2, 0.2), (0.5, 0.2, 0.2), (0.3, 0.5, 0.2), (0.3, 0.2, 0.2), (0.3, 0.3, 0.1)]
import random
for y in range(600):
    print("Iteration ={}".format(y))

    a = random.uniform(0.3, 0.8)
    b = random.uniform(0.2, 0.7)
    c = random.uniform(0.1, 0.25)

    print("a is= {} ,b is= {}, c is= {}".format(a,b,c))
    Hrecommender.fit(a,b,c)
    result = Hrecommender.evaluateRecommendations(URM_test)
    print("Recommender MAP is= {}".format(result["MAP"]))

Iteration =0
a is= 0.6808852884371784 ,b is= 0.44207001213756014, c is= 0.13432341662195385
SequentialEvaluator: Processed 9676 ( 100.00% ) in 11.44 seconds. Users per second: 846
Recommender MAP is= 0.0868673967805319
Iteration =1
a is= 0.7909935083050987 ,b is= 0.30488005874534324, c is= 0.12395506205204583
SequentialEvaluator: Processed 9676 ( 100.00% ) in 15.95 seconds. Users per second: 607
Recommender MAP is= 0.08631595980165509
Iteration =2
a is= 0.5729267540781127 ,b is= 0.5859592878786771, c is= 0.20884475927472979
SequentialEvaluator: Processed 9676 ( 100.00% ) in 14.32 seconds. Users per second: 676
Recommender MAP is= 0.08543773671335232
Iteration =3
a is= 0.40452050233952075 ,b is= 0.5877081637777397, c is= 0.11447973236154264
SequentialEvaluator: Processed 9676 ( 100.00% ) in 12.43 seconds. Users per second: 779
Recommender MAP is= 0.08710877621052097
Iteration =4
a is= 0.39621729833213165 ,b is= 0.4934672370053043, c is= 0.2151965673997079
SequentialEvaluator: Processed 

SequentialEvaluator: Processed 9676 ( 100.00% ) in 11.15 seconds. Users per second: 868
Recommender MAP is= 0.07932207097122361
Iteration =38
a is= 0.753524570193911 ,b is= 0.4180884381982203, c is= 0.19507558299295047
SequentialEvaluator: Processed 9676 ( 100.00% ) in 11.41 seconds. Users per second: 848
Recommender MAP is= 0.08488919361916672
Iteration =39
a is= 0.6462520326186019 ,b is= 0.36526943151834457, c is= 0.17081123165242013
SequentialEvaluator: Processed 9676 ( 100.00% ) in 11.01 seconds. Users per second: 879
Recommender MAP is= 0.08494036589935683
Iteration =40
a is= 0.478493590060272 ,b is= 0.44823963409295575, c is= 0.16789495926415665
SequentialEvaluator: Processed 9676 ( 100.00% ) in 10.82 seconds. Users per second: 895
Recommender MAP is= 0.08525284339196144
Iteration =41
a is= 0.7621162383163674 ,b is= 0.29931386699817925, c is= 0.15943862445032303
SequentialEvaluator: Processed 9676 ( 100.00% ) in 11.03 seconds. Users per second: 877
Recommender MAP is= 0.084938562

Recommender MAP is= 0.08060642254563077
Iteration =75
a is= 0.3279471585695479 ,b is= 0.6821473192800014, c is= 0.19043852317510657
SequentialEvaluator: Processed 9676 ( 100.00% ) in 16.03 seconds. Users per second: 603
Recommender MAP is= 0.0856653283212829
Iteration =76
a is= 0.6299803434104772 ,b is= 0.3927559556776996, c is= 0.2403426076117807
SequentialEvaluator: Processed 9676 ( 100.00% ) in 16.15 seconds. Users per second: 599
Recommender MAP is= 0.08264314122426766
Iteration =77
a is= 0.7588346857416964 ,b is= 0.293002512823406, c is= 0.13158935580940462
SequentialEvaluator: Processed 9676 ( 100.00% ) in 16.07 seconds. Users per second: 602
Recommender MAP is= 0.0858857551075869
Iteration =78
a is= 0.7238154118160998 ,b is= 0.5151671906232724, c is= 0.21046787380545365
SequentialEvaluator: Processed 9676 ( 100.00% ) in 18.17 seconds. Users per second: 533
Recommender MAP is= 0.08518874680554463
Iteration =79
a is= 0.6218186650156914 ,b is= 0.3723183729167917, c is= 0.1819284128

SequentialEvaluator: Processed 9676 ( 100.00% ) in 15.62 seconds. Users per second: 619
Recommender MAP is= 0.08742776748654375
Iteration =113
a is= 0.6645762587663224 ,b is= 0.4507883707657125, c is= 0.14296476335067335
SequentialEvaluator: Processed 9676 ( 100.00% ) in 15.27 seconds. Users per second: 634
Recommender MAP is= 0.08657314982245103
Iteration =114
a is= 0.7527167835527606 ,b is= 0.5558611258371225, c is= 0.11039600103108571
SequentialEvaluator: Processed 9676 ( 100.00% ) in 14.88 seconds. Users per second: 650
Recommender MAP is= 0.08771716336045247
Iteration =115
a is= 0.40242555127930374 ,b is= 0.5685630264670242, c is= 0.19697369870068043
SequentialEvaluator: Processed 9676 ( 100.00% ) in 14.93 seconds. Users per second: 648
Recommender MAP is= 0.08481859307902567
Iteration =116
a is= 0.6836359137255037 ,b is= 0.6370197808150614, c is= 0.1384064276066716
SequentialEvaluator: Processed 9676 ( 100.00% ) in 17.49 seconds. Users per second: 553
Recommender MAP is= 0.087411

Recommender MAP is= 0.080417122038632
Iteration =150
a is= 0.3706865106594747 ,b is= 0.46282915609946207, c is= 0.12939505324016456
SequentialEvaluator: Processed 9676 ( 100.00% ) in 15.01 seconds. Users per second: 645
Recommender MAP is= 0.08604920963857343
Iteration =151
a is= 0.7504172243756904 ,b is= 0.6109796694214203, c is= 0.10401823697380547
SequentialEvaluator: Processed 9676 ( 100.00% ) in 16.20 seconds. Users per second: 597
Recommender MAP is= 0.0875559889929821
Iteration =152
a is= 0.6766770508432808 ,b is= 0.5325456890464527, c is= 0.2379488267457456
SequentialEvaluator: Processed 9676 ( 100.00% ) in 15.26 seconds. Users per second: 634
Recommender MAP is= 0.08441576341543615
Iteration =153
a is= 0.6779472797069188 ,b is= 0.3397361861255087, c is= 0.1364707948192292
SequentialEvaluator: Processed 9676 ( 100.00% ) in 15.01 seconds. Users per second: 645
Recommender MAP is= 0.08600470193972264
Iteration =154
a is= 0.7740609931950999 ,b is= 0.5483358351769214, c is= 0.16700

In [ ]:
import pandas as pd
from numpy import *
import numpy as np
from scipy import sparse



train_df= pd.read_csv('all/train.csv',low_memory = False)
target_playlist_df= pd.read_csv('all/target_playlists.csv',low_memory = False)

random_playlist_df=target_playlist_df.iloc[5000:10000]
sequential_playlist_df=target_playlist_df.iloc[:5000]


#-----CALCOLO DF per TEST
#prendo il 20% delle random playlist da target playlist
random_id = random_playlist_df.playlist_id.values
random_data=train_df.loc[train_df['playlist_id'].isin(random_id)]
#random_data.shape = (114513, 2)
n=round((114513/100)*20)
random_test_df = random_data.sample(n=n)

#prendo il 20% delle sequential playlist da target playlist
sequential_id = sequential_playlist_df.playlist_id.values
sequential_data=train_df.loc[train_df['playlist_id'].isin(sequential_id)]
n1=round((115553/100)*20)
for pid in sequential_id:
    playlistID=sequential_data.loc[sequential_data['playlist_id']==pid]
    start=sequential_data.index[sequential_data['playlist_id'] == pid][0]
    end=start+round((playlistID.shape[0]*50)/100)

    dropping=list(range(start, end))    
    sequential_data=sequential_data.drop(dropping)


sequential_test_df = sequential_data.sample(n=n1)

In [ ]:
#SETTO URM TEST
URM_test_sequential=zeros((50446,20635))
URM_test_random=zeros((50446,20635))



#20% from random
for row in random_test_df.itertuples():
    riga = row.playlist_id
    #print(riga)
    colonna= row.track_id
    #print(colonna)
    URM_test_random[riga,colonna]=1
#20% from random
for row in sequential_test_df.itertuples():
    riga = row.playlist_id
    #print(riga)
    colonna= row.track_id
    #print(colonna)
    URM_test_sequential[riga,colonna]=1

URM_test_sequential=sparse.csr_matrix(URM_test_sequential) 
URM_test_random=sparse.csr_matrix(URM_test_random) 

In [ ]:
print("CF Recommender")
result = CFrecommender.evaluateRecommendations(URM_test_sequential)
print("Recommender MAP is= {}".format(result["MAP"]))
result = CFrecommender.evaluateRecommendations(URM_test_random)
print("Recommender MAP is= {}".format(result["MAP"]))

print("CBF Recommender")
result = CBFrecommender.evaluateRecommendations(URM_test_sequential)
print("Recommender MAP is= {}".format(result["MAP"]))
result = CBFrecommender.evaluateRecommendations(URM_test_random)
print("Recommender MAP is= {}".format(result["MAP"]))

print("SLIM Recommender")
result = SLIMrecommender.evaluateRecommendations(URM_test_sequential)
print("Recommender MAP is= {}".format(result["MAP"]))
result = SLIMrecommender.evaluateRecommendations(URM_test_random)
print("Recommender MAP is= {}".format(result["MAP"]))

print("SLIME Recommender")

result = SLIMErecommender.evaluateRecommendations(URM_test_sequential)
print("Recommender MAP is= {}".format(result["MAP"]))
result = SLIMErecommender.evaluateRecommendations(URM_test_random)
print("Recommender MAP is= {}".format(result["MAP"]))

print("UCF Recommender")
result = UCFRecommender.evaluateRecommendations(URM_test_sequential)
print("Recommender MAP is= {}".format(result["MAP"]))
result = UCFRecommender.evaluateRecommendations(URM_test_random)
print("Recommender MAP is= {}".format(result["MAP"]))

In [ ]:
superSlim=ItemKNNSimilarityHybridRecommender(URM_train,SLIMrecommender,SLIMErecommender)
superSlim.fit(topK=600, alpha = 0.5)

hybrid1=ItemKNNSimilarityHybridRecommender3(URM_train,superSlim, CFrecommender,CBFrecommender)
hybrid1.fit(topK=800, alpha=0.7, beta=0.4, gamma=0.2)

In [ ]:
tmpRec=ItemKNNSimilarityHybridRecommender3(URM_train,superSlim, CFrecommender,CBFrecommender)
tmpRec.fit(topK=1000,alpha=0.6,beta=0.2,gamma=0.2)
hybrid2=ItemKNNScoresHybridRecommender(URM_train,tmpRec,UCFRecommender)
hybrid2.fit(alpha=0.7)

In [ ]:
hybrid3 = ItemKNNSimilarityHybridRecommender4(URM_train, SLIMrecommender, SLIMErecommender, CFrecommender,CBFrecommender)
hybrid3.fit(topK=400, alpha=0.486, beta=0.4547, gamma=0.4198, delta=0.255)

In [ ]:
print("Hybrid1 Recommender")
result = hybrid1.evaluateRecommendations(URM_test_sequential)
print("Recommender MAP is= {}".format(result["MAP"]))
result = hybrid1.evaluateRecommendations(URM_test_random)
print("Recommender MAP is= {}".format(result["MAP"]))

print("Hybrid2 Recommender")

result = hybrid2.evaluateRecommendations(URM_test_sequential)
print("Recommender MAP is= {}".format(result["MAP"]))
result = hybrid2.evaluateRecommendations(URM_test_random)
print("Recommender MAP is= {}".format(result["MAP"]))

print("Hybrid3 Recommender")
result = hybrid3.evaluateRecommendations(URM_test_sequential)
print("Recommender MAP is= {}".format(result["MAP"]))
result = hybrid3.evaluateRecommendations(URM_test_random)
print("Recommender MAP is= {}".format(result["MAP"]))

In [ ]:
hybrid4=ItemKNNScoresHybridRecommender(URM_train,superSlim,UCFRecommender)
hybrid4.fit(alpha=0.8)
hybrid5=ItemKNNSimilarityHybridRecommender3(URM_train,SLIMrecommender, CFrecommender,CBFrecommender)
hybrid5.fit(topK=800, alpha=0.7, beta=0.5, gamma=0.2)

print("Hybrid4 Recommender")
result = hybrid4.evaluateRecommendations(URM_test_sequential)
print("Recommender MAP is= {}".format(result["MAP"]))
result = hybrid4.evaluateRecommendations(URM_test_random)
print("Recommender MAP is= {}".format(result["MAP"]))

print("Hybrid5 Recommender")
result = hybrid5.evaluateRecommendations(URM_test_sequential)
print("Recommender MAP is= {}".format(result["MAP"]))
result = hybrid5.evaluateRecommendations(URM_test_random)
print("Recommender MAP is= {}".format(result["MAP"]))